In [ ]:
pip install kaleido
import os
import sys
import subprocess
import importlib.util
import argparse
import shutil
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_absolute_error, mean_squared_error

# ----------------------------------------
# Module: utils
# ----------------------------------------
def install_requirements(modules_string):
    modules = modules_string.split()
    for module in modules:
        if importlib.util.find_spec(module) is None:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", module])
                print(f"Successfully installed {module}.")
            except subprocess.CalledProcessError as e:
                print(f"An error occurred while installing {module}: {e}")
                sys.exit(1)

def parse_args():
    parser = argparse.ArgumentParser(description="Stock Analysis Script")
    parser.add_argument('ticker', type=str, help='Stock ticker symbol (e.g., AAPL)')
    parser.add_argument('start_date', type=str, help='Start date for stock data (format: YYYY-MM-DD)')
    parser.add_argument('end_date', type=str, help='End date for stock data (format: YYYY-MM-DD)')
    return parser.parse_args()

# ----------------------------------------
# Module: data
# ----------------------------------------
def load_data(ticker, start_date, end_date):
    df = pd.DataFrame(ticker.history(start=start_date, end=end_date))
    df.index = pd.to_datetime(df.index.strftime(r'%Y/%m/%d'))
    return df

def data_split(df):
    df_prices = df[['Close','Volume']]
    if 'Stock Splits' in df.columns:
        df_actions = df[['Dividends', 'Stock Splits','Close']]
    else:
        df_actions = df[['Dividends', 'Close']]
    return df_prices, df_actions

def fill_data(df):
    df.index = pd.to_datetime(df.index.strftime(r'%Y/%m/%d'))
    df = df.reset_index().rename(columns={'index': 'Date'})
    full_date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='D')
    df = df.set_index('Date').reindex(full_date_range)
    for col in df.columns:
        df[col] = df[col].interpolate()
    return df

# ----------------------------------------
# Module: visualizations
# ----------------------------------------
def plot_df_prices(df):
    fig_prices = go.Figure()
    fig_vol = go.Figure()
    fig_prices.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close'))
    fig_vol.add_trace(go.Scatter(x=df.index, y=df['Volume'], mode='lines', name='Volume'))
    return fig_prices, fig_vol

def volatility(df):
    df['Daily_Return'] = df['Close'].pct_change()
    df['Volatility_30'] = df['Daily_Return'].rolling(window=30).std()
    df['Volatility_90'] = df['Daily_Return'].rolling(window=90).std()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['Volatility_30'], mode='lines', name='30-Day Volatility', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df.index, y=df['Volatility_90'], mode='lines', name='90-Day Volatility', line=dict(color='red')))
    fig.update_layout(title='Stock Volatility Over Time', xaxis_title='Date', yaxis_title='Volatility')
    return fig

def seasonal_decomposition_plot(df):
    result = seasonal_decompose(df['Close'], model='additive', period=90)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=result.trend, mode='lines', name='Trend'))
    fig.add_trace(go.Scatter(x=df.index, y=result.seasonal, mode='lines', name='Seasonal'))
    fig.add_trace(go.Scatter(x=df.index, y=result.resid, mode='lines', name='Residual'))
    fig.update_layout(title='Seasonal Decomposition of Stock Price', xaxis_title='Date', yaxis_title='Price')
    return fig

def dicky_fuller_test(df):
    close = df['Close']
    differenced_close = close.diff().dropna()
    dicftest = adfuller(close)
    diff_dicftest = adfuller(differenced_close)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=differenced_close.index, y=differenced_close.values, mode='lines', name='Differenced Close Prices'))
    fig.update_layout(title="Differenced Close Prices", xaxis_title="Date", yaxis_title="Differenced Close Price")
    return dicftest, diff_dicftest, fig

def plot_moving_averages(df):
    close = df['Close']
    ma30 = close.rolling(window=30).mean()
    ma90 = close.rolling(window=90).mean()
    ma120 = close.rolling(window=120).mean()
    ma365 = close.rolling(window=365).mean()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=ma30, mode='lines', name='30-Day MA'))
    fig.add_trace(go.Scatter(x=df.index, y=ma90, mode='lines', name='90-Day MA'))
    fig.add_trace(go.Scatter(x=df.index, y=ma120, mode='lines', name='120-Day MA'))
    fig.add_trace(go.Scatter(x=df.index, y=ma365, mode='lines', name='365-Day MA'))
    fig.update_layout(title='Moving Averages of Close Price', xaxis_title='Date', yaxis_title='Price')
    return fig

def dividend_yield_plot(df):
    df['Yield'] = df['Dividends'] / df['Close']
    df_filtered = df[df['Dividends'] != 0]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_filtered.index, y=df_filtered['Yield'], mode='markers', name='Yield'))
    fig.update_layout(title='Dividend Yield Over Time', xaxis_title='Date', yaxis_title='Yield', template='plotly_dark')
    return fig

# ----------------------------------------
# Module: dlmodels
# ----------------------------------------
def dl_output(df):
    def metric_funcs(actual, pred):
        def MAPE(actual, pred):
            return mean_absolute_percentage_error(actual, pred)
        def R2(actual, pred):
            return r2_score(actual, pred)
        def MAE(actual, pred):
            return mean_absolute_error(actual, pred)
        def RMSE(actual, pred):
            return np.sqrt(mean_squared_error(actual, pred))
        return MAPE, R2, MAE, RMSE

    data = df.filter(['Close'])
    dataset = data.values
    training_data_len = int(np.ceil(len(dataset) * 0.90))
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)

    train_data = scaled_data[:training_data_len, :]

    x_train, y_train = [], []
    for i in range(90, len(train_data)):
        x_train.append(train_data[i-90:i, 0])
        y_train.append(train_data[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

    def rnn_model(model_type):
        model = Sequential()
        if model_type == 'GRU':
            model.add(GRU(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
            model.add(GRU(64, return_sequences=False))
        elif model_type == 'LSTM':
            model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
            model.add(LSTM(64, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=1, verbose=0)

        test_data = scaled_data[training_data_len - 90:]
        x_test = []
        y_test = dataset[training_data_len:]
        for i in range(90, len(test_data)):
            x_test.append(test_data[i-90:i, 0])
        x_test = np.array(x_test).reshape(len(x_test), 90, 1)
        predictions = model.predict(x_test)
        predictions = scaler.inverse_transform(predictions)

        MAPE, R2, MAE, RMSE = metric_funcs(y_test, predictions)
        metrics = {
            'RMSE': RMSE(y_test, predictions),
            'MAPE': MAPE(y_test, predictions),
            'MAE': MAE(y_test, predictions),
            'R2': R2(y_test, predictions)
        }

        train = data[:training_data_len]
        valid = data[training_data_len:].copy()
        valid['Predictions'] = predictions
        trace_train = go.Scatter(x=train.index, y=train['Close'], mode='lines', name='Train', line=dict(color='blue'))
        trace_valid = go.Scatter(x=valid.index, y=valid['Close'], mode='lines', name='Actual', line=dict(color='blue'))
        trace_pred = go.Scatter(x=valid.index, y=valid['Predictions'], mode='lines', name='Predicted', line=dict(color='red', dash='dash'))
        fig = go.Figure(data=[trace_train, trace_valid, trace_pred])
        fig.update_layout(title=f'{model_type} Model Predictions', xaxis_title='Date', yaxis_title='Close Price')
        return metrics, fig, predictions, y_test

    return {
        'LSTM': rnn_model('LSTM'),
        'GRU': rnn_model('GRU')
    }

# ----------------------------------------
# Module: evaluation
# ----------------------------------------
def calculate_residuals(actual, predicted):
    residuals = actual - predicted
    return residuals

def plot_residuals(residuals):
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=residuals, nbinsx=50))
    fig.update_layout(
        title="Residuals Histogram",
        xaxis_title="Residual",
        yaxis_title="Count"
    )
    return fig

def theils_u(actual, predicted):
    actual = np.array(actual)
    predicted = np.array(predicted)
    naive_forecast = np.roll(actual, 1)
    naive_forecast[0] = actual[0]
    mse_model = np.mean((predicted - actual)**2)
    mse_naive = np.mean((naive_forecast - actual)**2)
    u = np.sqrt(mse_model / mse_naive)
    return u

def plot_visual_comparison(actual, predicted):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=np.arange(len(actual)),
        y=actual,
        mode='lines',
        name='Actual',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=np.arange(len(predicted)),
        y=predicted,
        mode='lines',
        name='Predicted',
        line=dict(color='red', dash='dash')
    ))
    fig.update_layout(
        title="Visual Comparison: Actual vs. Predicted",
        xaxis_title="Time",
        yaxis_title="Value"
    )
    return fig

def time_series_cross_validation(model_func, series, initial_train_size, forecast_horizon, step_size):
    n = len(series)
    predictions = []
    actuals = []
    for start in range(initial_train_size, n - forecast_horizon + 1, step_size):
        train = series[:start]
        test = series[start:start + forecast_horizon]
        forecast = model_func(train, forecast_horizon)
        predictions.extend(forecast)
        actuals.extend(test)
    return np.array(actuals), np.array(predictions)

def backtesting_forecast(model_func, series, forecast_horizon, rolling_window):
    n = len(series)
    predictions = []
    forecast_times = []
    for start in range(0, n - forecast_horizon, rolling_window):
        train = series[:start + forecast_horizon]
        forecast = model_func(train, forecast_horizon)
        predictions.append(forecast[0])  # Assuming forecast_horizon=1 for simplicity
        forecast_times.append(start + forecast_horizon)
    predictions = np.array(predictions)
    actual = series[forecast_times]
    return forecast_times, actual, predictions

def naive_forecast(train, forecast_horizon):
    last_value = train[-1]
    return [last_value] * forecast_horizon

# ----------------------------------------
# Helper: save figure with fallback
# ----------------------------------------
def save_figure(fig, png_path, html_path):
    try:
        fig.write_image(png_path)
        return png_path  # Return PNG path if successful
    except Exception as e:
        print(f"Error saving {png_path} using kaleido: {e}\nFalling back to HTML export.")
        fig.write_html(html_path)
        return html_path  # Return HTML path as fallback

# ----------------------------------------
# Main code
# ----------------------------------------
# Check if running in Google Colab
try:
    from google.colab import files
    is_collab = True
except ImportError:
    is_collab = False

def main():
    # Ensure required packages are installed (kaleido, plotly, etc.)
    install_requirements("yfinance pandas numpy plotly statsmodels scikit-learn keras kaleido")
    
    # For Colab testing, use default parameters if no command-line arguments are provided.
    if len(sys.argv) < 4:
        ticker_symbol = "AAPL"
        start_date = "2020-01-01"
        end_date = "2021-01-01"
        print("No command-line arguments provided. Using default ticker AAPL and date range 2020-01-01 to 2021-01-01")
    else:
        args = parse_args()
        ticker_symbol, start_date, end_date = args.ticker, args.start_date, args.end_date

    ticker = yf.Ticker(ticker_symbol.upper())
    full_df = load_data(ticker, start_date, end_date)
    df_prices, df_actions = data_split(full_df)
    df_filled = fill_data(df_prices)

    fig_prices, fig_volume = plot_df_prices(df_filled)
    vol_fig = volatility(df_filled)
    season_fig = seasonal_decomposition_plot(df_filled)
    dicky_results = dicky_fuller_test(df_filled)
    dicky_fig = dicky_results[2]
    ma_fig = plot_moving_averages(df_filled)
    dy_fig = dividend_yield_plot(df_actions)

    dl_results = dl_output(df_filled)
    lstm_metrics, lstm_fig, lstm_predictions, actual_values = dl_results['LSTM']
    gru_metrics, gru_fig, gru_predictions, _ = dl_results['GRU']

    u_stat = theils_u(actual_values, lstm_predictions)

    series = df_filled['Close'].values.flatten()
    initial_train_size = int(0.7 * len(series))
    forecast_horizon = 1
    step_size = 10
    actual_cv, predictions_cv = time_series_cross_validation(naive_forecast, series, initial_train_size, forecast_horizon, step_size)
    cv_fig = plot_visual_comparison(actual_cv, predictions_cv)
    cv_fig.update_layout(title="Naive Forecast Cross-Validation")
    
    rolling_window = 10
    forecast_times, actual_bt, predictions_bt = backtesting_forecast(naive_forecast, series, forecast_horizon, rolling_window)
    bt_fig = plot_visual_comparison(actual_bt, predictions_bt)
    bt_fig.update_layout(title="Naive Forecast Backtesting")

    # Create directory for saving plots
    os.makedirs("plots", exist_ok=True)

    # Save figures using the helper function (PNG if possible, otherwise HTML)
    paths = {}
    paths['prices'] = save_figure(fig_prices, "plots/prices.png", "plots/prices.html")
    paths['volume'] = save_figure(fig_volume, "plots/volume.png", "plots/volume.html")
    paths['volatility'] = save_figure(vol_fig, "plots/volatility.png", "plots/volatility.html")
    paths['seasonal'] = save_figure(season_fig, "plots/seasonal_decomposition.png", "plots/seasonal_decomposition.html")
    paths['dickey'] = save_figure(dicky_fig, "plots/dickey_fuller.png", "plots/dickey_fuller.html")
    paths['moving_avg'] = save_figure(ma_fig, "plots/moving_averages.png", "plots/moving_averages.html")
    paths['dividend'] = save_figure(dy_fig, "plots/dividend_yield.png", "plots/dividend_yield.html")
    paths['lstm'] = save_figure(lstm_fig, "plots/lstm_predictions.png", "plots/lstm_predictions.html")
    paths['gru'] = save_figure(gru_fig, "plots/gru_predictions.png", "plots/gru_predictions.html")
    paths['cv'] = save_figure(cv_fig, "plots/cross_validation.png", "plots/cross_validation.html")
    paths['bt'] = save_figure(bt_fig, "plots/backtesting.png", "plots/backtesting.html")

    report_lines = []

    # 1. Title and Introduction
    report_lines.append(f"# Stock Analysis Report: {ticker_symbol.upper()}")
    report_lines.append("")
    report_lines.append("## 1. Introduction and Overview")
    report_lines.append(f"This report presents an analysis of stock data for the ticker symbol **{ticker_symbol.upper()}** from **{start_date}** to **{end_date}**. "
                        "It covers data loading, visualization, deep learning forecasts, and forecast evaluation metrics.")
    report_lines.append("")

    # 2. Detailed Definitions and Explanations
    report_lines.append("## 2. Definitions and Explanations")
    report_lines.append("### 2.1 Basic Stock Metrics")
    report_lines.append("- **Close Price**: The final trading price of the stock for the day. This value is critical for gauging market sentiment at the end of trading sessions.")
    report_lines.append("- **Volume**: The total number of shares traded during a given period. High volume often signals increased market interest or volatility.")
    report_lines.append("")
    report_lines.append("### 2.2 Time Series and Statistical Analysis")
    report_lines.append("- **Volatility**: A measure of how much the stock price fluctuates over time, calculated over intervals such as 30-day and 90-day periods.")
    report_lines.append("- **Seasonal Decomposition**: A process that splits a time series into trend, seasonal, and residual components to better understand underlying patterns.")
    report_lines.append("- **Dickey-Fuller Test**: A statistical test used to determine if a time series is stationary, which is a necessary condition for many forecasting models.")
    report_lines.append("")
    report_lines.append("### 2.3 Technical Indicators")
    report_lines.append("- **Moving Averages**: Techniques that smooth out short-term fluctuations to reveal longer-term trends. Common periods include 30, 90, 120, and 365 days.")
    report_lines.append("- **Dividend Yield**: The ratio of a company’s annual dividend relative to its share price, used to assess the income potential of an investment.")
    report_lines.append("")
    report_lines.append("### 2.4 Forecasting Models and Evaluation Metrics")
    report_lines.append("- **LSTM (Long Short-Term Memory)**: A type of recurrent neural network that captures long-term dependencies, used here to forecast closing prices.")
    report_lines.append("- **GRU (Gated Recurrent Unit)**: A streamlined version of LSTM with fewer parameters, offering similar forecasting capabilities with improved computational efficiency.")
    report_lines.append("- **Theil's U Statistic**: A metric that compares the performance of the forecast model to a naive forecast. Lower values indicate better performance.")
    report_lines.append("- **Naive Forecast**: A baseline forecasting method that simply carries forward the last observed value as the future prediction.")
    report_lines.append("- **Cross-Validation**: A method to assess model generalizability by partitioning data into training and validation sets.")
    report_lines.append("- **Backtesting**: The process of evaluating a forecasting model using historical data to simulate its performance in real-world scenarios.")
    report_lines.append("")

    # 3. Data Loading and Preparation
    report_lines.append("## 3. Data Loading and Preparation")
    report_lines.append(f"The dataset was obtained from Yahoo Finance for the ticker symbol **{ticker_symbol.upper()}** between **{start_date}** and **{end_date}**. "
                        "Missing values were filled using interpolation to create a continuous time series for analysis.")
    report_lines.append("")

    # 4. Visualizations and Detailed Results
    report_lines.append("## 4. Visualizations and Detailed Results")
    report_lines.append("")
    # 4.1 Stock Prices
    report_lines.append("### 4.1 Stock Prices")
    report_lines.append("**Description**: This plot displays the daily closing prices over the analysis period.")
    report_lines.append("**Interpretation**: Fluctuations in the closing price indicate overall market trends, investor sentiment, and potential support/resistance levels.")
    report_lines.append("![Stock Prices](plots/prices.png)")
    report_lines.append("")
    # 4.2 Trading Volume
    report_lines.append("### 4.2 Trading Volume")
    report_lines.append("**Description**: This chart shows the volume of shares traded each day.")
    report_lines.append("**Interpretation**: High trading volume may signal increased market interest or impending price movements.")
    report_lines.append("![Trading Volume](plots/volume.png)")
    report_lines.append("")
    # 4.3 Volatility
    report_lines.append("### 4.3 Volatility")
    report_lines.append("**Description**: This visualization illustrates the stock's volatility over 30-day and 90-day intervals.")
    report_lines.append("**Interpretation**: Volatility is a key risk metric; higher volatility implies greater uncertainty and risk.")
    report_lines.append("![Volatility](plots/volatility.png)")
    report_lines.append("")
    # 4.4 Seasonal Decomposition
    report_lines.append("### 4.4 Seasonal Decomposition")
    report_lines.append("**Description**: The seasonal decomposition plot breaks the stock price data into trend, seasonal, and residual components.")
    report_lines.append("**Interpretation**: This helps identify underlying patterns, cyclical behavior, and irregular fluctuations.")
    report_lines.append("![Seasonal Decomposition](plots/seasonal_decomposition.png)")
    report_lines.append("")
    # 4.5 Dickey-Fuller Test
    report_lines.append("### 4.5 Dickey-Fuller Test (Differenced Close Prices)")
    report_lines.append("**Description**: This plot shows the differenced close prices used for assessing the stationarity of the time series.")
    report_lines.append("**Interpretation**: A stationary time series is crucial for reliable forecasting; the Dickey-Fuller test helps determine stationarity.")
    report_lines.append("![Dickey-Fuller Test](plots/dickey_fuller.png)")
    report_lines.append("")
    # 4.6 Moving Averages
    report_lines.append("### 4.6 Moving Averages")
    report_lines.append("**Description**: This plot overlays several moving averages (30, 90, 120, and 365 days) on the closing price data.")
    report_lines.append("**Interpretation**: Moving averages smooth out short-term fluctuations, clarifying the overall trend in stock prices.")
    report_lines.append("![Moving Averages](plots/moving_averages.png)")
    report_lines.append("")
    # 4.7 Dividend Yield
    report_lines.append("### 4.7 Dividend Yield")
    report_lines.append("**Description**: This visualization displays the dividend yield over time.")
    report_lines.append("**Interpretation**: Dividend yield is an important metric for income-focused investors, indicating the cash return on investment relative to the stock price.")
    report_lines.append("![Dividend Yield](plots/dividend_yield.png)")
    report_lines.append("")

    # 5. Deep Learning Model Forecasts
    report_lines.append("## 5. Deep Learning Model Forecasts")
    report_lines.append("The deep learning models (LSTM and GRU) were trained on 90% of the data to forecast the stock's closing price.")
    report_lines.append("")
    # 5.1 LSTM Model
    report_lines.append("### 5.1 LSTM Model Forecast")
    report_lines.append("**Metrics:**")
    for key, value in lstm_metrics.items():
        report_lines.append(f"- **{key}**: {value:.4f}")
    report_lines.append("**Visualization:** The plot below compares the LSTM model's predictions to the actual stock prices.")
    report_lines.append("![LSTM Predictions](plots/lstm_predictions.png)")
    report_lines.append("")
    # 5.2 GRU Model
    report_lines.append("### 5.2 GRU Model Forecast")
    report_lines.append("**Metrics:**")
    for key, value in gru_metrics.items():
        report_lines.append(f"- **{key}**: {value:.4f}")
    report_lines.append("**Visualization:** The plot below shows the GRU model's predictions alongside the actual values.")
    report_lines.append("![GRU Predictions](plots/gru_predictions.png)")
    report_lines.append("")

    # 6. Forecast Evaluation
    report_lines.append("## 6. Forecast Evaluation")
    # 6.1 Theil's U Statistic
    report_lines.append("### 6.1 Theil's U Statistic")
    report_lines.append(f"- **Theil's U**: {u_stat:.4f}")
    report_lines.append("**Interpretation:** A lower Theil's U value indicates that the forecasting model performs better than the naive forecast.")
    report_lines.append("")
    # 6.2 Cross-Validation
    report_lines.append("### 6.2 Cross-Validation (Naive Forecast)")
    report_lines.append("**Description:** This plot shows the results of cross-validation using a naive forecasting approach.")
    report_lines.append("**Interpretation:** Cross-validation assesses the consistency of the naive forecast across different data segments.")
    report_lines.append("![Cross-Validation](plots/cross_validation.png)")
    report_lines.append("")
    # 6.3 Backtesting Forecast
    report_lines.append("### 6.3 Backtesting Forecast (Naive Forecast)")
    report_lines.append("**Description:** The backtesting chart evaluates forecast performance using a rolling window on historical data.")
    report_lines.append("**Interpretation:** Backtesting helps verify that the forecast model's performance is robust over time.")
    report_lines.append("![Backtesting Forecast](plots/backtesting.png)")
    report_lines.append("")

    # 7. Conclusion
    report_lines.append("## 7. Conclusion")
    report_lines.append("This report summarizes the key findings from the stock analysis. The detailed visualizations reveal important trends and patterns in the stock's performance, while the deep learning models offer forecasts benchmarked against naive methods. "
                        "The comprehensive definitions and explanations provided throughout this report are intended to assist in interpreting the analysis results and making informed investment decisions.")

    report_content = "\n".join(report_lines)

    with open("report.md", "w") as f:
        f.write(report_content)

    print("Markdown report generated as report.md.")

    shutil.make_archive("plots", 'zip', "plots")
    print("Created plots.zip containing the image files.")

    if is_collab:
        from google.colab import files
        files.download("report.md")
        files.download("plots.zip")

if __name__ == "__main__":
    main()
